In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from os.path import join as oj
import sys
sys.path.append('../src')
# sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import numpy as np
import pandas as pd
import data

# load data
df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = data.load_it_all(dummy=False, impute=True, frac_missing_allowed=1)
d = df_pecarn # [common_feats]
d['Age<2'] = d['Age'] < 2
# data.select_final_feats(common_feats)

/accounts/projects/vision/.local/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


# check bivariable associations

In [2]:
# can also check vars that were not used
rules = [
    ('AbdTrauma_or_SeatBeltSign', ['yes']),
    ('GCSScore', range(14)),
#     ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
    ('AbdTenderDegree', ['None']),    
    ('AbdTenderDegree', ['Mild']),    
    ('AbdTenderDegree', ['Moderate']),    
    ('AbdTenderDegree', ['Severe']),        
    ('ThoracicTrauma', ['yes']),        
    ('AbdomenPain', ['yes']),
    ('DecrBreathSound', ['yes']),
    ('VomitWretch', ['yes']),
    
    # not used in rule
    ('Age<2', [True]),    
    ('AbdDistention', ['yes']),
    ('MOI', ['Motor vehicle collision']),
    ('DistractingPain', ['yes']),    
    ('CostalTender', ['yes']),
    ('ThoracicTender', ['yes']),    
    ('Hypotension', ['yes']),        
    ('AbdomenTender', [1]),    
]
index = ['NA', 'IAIw/I', 'No IAIw/I', 'Difference']
o = data.outcome_def
r = {k + ' = ' + str(v[0]) : [] for k, v in rules}
io = d[o] == 1
print('num IwI and key / num IwI (not nan) \t num without IwI and key / num without IwI (not nane) ')
for rule in rules:
    k, vals = rule
    kk = k + ' = ' + str(vals[0])
    ik = d[k].isin(vals) # d[o]==1
    inan = d[k].isna() | (d[k] == 'unknown')
    r[kk].append(np.sum(inan))   
    num1 = np.sum(d[ik][o]==1)
    denom1 = (io & ~inan).sum()
    r[kk].append(f'{num1}/{denom1}')
    num2 = np.sum(d[ik][o]==0)
    denom2 = (~io & ~inan).sum()
    r[kk].append(f'{num2}/{denom2}')
    r[kk].append(f'{num1/denom1 - num2/denom2:0.3f}')    
r = pd.DataFrame.from_dict(r)
r.index = index
r.transpose()

num IwI and key / num IwI (not nan) 	 num without IwI and key / num without IwI (not nane) 


,NA,IAIw/I,No IAIw/I,Difference
AbdTrauma_or_SeatBeltSign = yes,0,112/203,1851/11841,0.395
GCSScore = 0,0,67/203,928/11841,0.252
AbdTenderDegree = None,43,89/202,8344/11799,-0.267
AbdTenderDegree = Mild,43,12/202,1601/11799,-0.076
AbdTenderDegree = Moderate,43,42/202,1494/11799,0.081
AbdTenderDegree = Severe,43,59/202,360/11799,0.262
ThoracicTrauma = yes,76,66/201,1976/11767,0.160
AbdomenPain = yes,148,105/201,3515/11695,0.222
DecrBreathSound = yes,233,25/197,231/11614,0.107
VomitWretch = yes,472,49/165,1104/11407,0.200


# recreate / evaluate rule

In [15]:
def pecarn_rule_predict(d, o=data.outcome_def):
    '''Predict each subgroup in the original PECARN rule
    '''
    n = d.shape[0]
    npos = d[o].sum()
    print(f'{"Initial":<25} {npos} / {n}')
    risks = np.array([np.nan] * d.shape[0])
    rules = [
        ('AbdTrauma_or_SeatBeltSign', ['yes']),
        ('GCSScore', range(14)),
        ('AbdTenderDegree', ['Mild', 'Moderate', 'Severe']),
        ('ThoracicTrauma', ['yes']),        
        ('AbdomenPain', ['yes']),
        ('DecrBreathSound', ['yes']),
        ('VomitWretch', ['yes']),
    ]
    for rule in rules:
        k, vals = rule
        idxs = d[k].isin(vals)
        do = d[idxs]
        d = d[~idxs]
        num2 = do[o].sum()
        denom2 = do.shape[0]
        print(f'{k:<25} {d[o].sum():>3} / {d.shape[0]:>5}\t{num2:>3} / {denom2:>4} ({num2/denom2*100:0.1f})')
    
    low_risk_patients = d
    patients_missed = low_risk_patients[low_risk_patients[o] == 1]
    
    # calc metrics
    fn = patients_missed.shape[0]
    tp = npos - fn
    tn = low_risk_patients.shape[0] - fn
    fp = (n - low_risk_patients.shape[0]) - tp
    sens = tp / (tp + fn)
    spec = tn / (tn + fp)
    stats = {
        'fn': fn,
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'sensitivity': sens * 100,
        'specificity': spec * 100,
        'ppv': tp / (tp + fp),
        'npv': tn / (tn + fn),
        'lr+': sens / (1 - spec),
        'lr-': (1 - sens) / spec,
    }
    return d, patients_missed, stats

**look at pecarn patients**

In [17]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_pecarn)
assert missed_patients.shape[0] == 6, 'should find 6 low-risk patients'    
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')


# save out missed patients
df_pecarn.loc[missed_patients.index].to_csv('../reports/missed_patients_pecarn.csv')
print('\nmissed patients:')
missed_patients[['Age', 'Sex', 'MOI']]

Initial                   203 / 12044
AbdTrauma_or_SeatBeltSign  91 / 10081	112 / 1963 (5.7)
GCSScore                   53 /  9255	 38 /  826 (4.6)
AbdTenderDegree            17 /  6767	 36 / 2488 (1.4)
ThoracicTrauma             11 /  5805	  6 /  962 (0.6)
AbdomenPain                 9 /  5469	  2 /  336 (0.6)
DecrBreathSound             8 /  5435	  1 /   34 (2.9)
VomitWretch                 6 /  5040	  2 /  395 (0.5)
sensitivity 97.04
specificity 42.51

missed patients:


,Age,Sex,MOI
203,2,M,Fall down stairs
4349,2,M,Pedestrian/bicyclist struck by moving vehicle
5977,16,M,Motorcycle/ATV/Scooter collision
9419,17,F,Motor vehicle collision
9436,17,M,Motor vehicle collision
9971,17,M,Motor vehicle collision


**look at psrc patients**

In [ ]:
low_risk_patients, missed_patients, stats = pecarn_rule_predict(df_psrc)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')

# save out missed patients
df_psrc.loc[missed_patients.index].to_csv('../reports/missed_patients_psrc.csv')
missed_patients[['Age', 'MOI']]

In [19]:
stats

{'fn': 2,
 'tp': 60,
 'tn': 935,
 'fp': 1191,
 'sensitivity': 96.7741935483871,
 'specificity': 43.97930385700847,
 'ppv': 0.047961630695443645,
 'npv': 0.9978655282817502,
 'lr+': 1.72747217030958,
 'lr-': 0.07334828359496284}

# look at errors for subgroups
**here we consider the riskier subgroups (young age, female)**

In [12]:
# filter patients by age or sex
# d = df_pecarn[df_pecarn['Sex'] == 'F']
d = df_pecarn[df_pecarn['Age'] <= 15]
# d = df_pecarn[df_pecarn['Age'] <= 2]


low_risk_patients, missed_patients, stats = pecarn_rule_predict(d)
for stat in ['sensitivity', 'specificity']:
    print(stat, f'{stats[stat]:0.2f}')

print('\nmissed patients:')
missed_patients[['Age', 'Sex', 'MOI']]

Initial                   166 / 9907
AbdTrauma_or_SeatBeltSign  74 /  8226	 92 / 1681 (5.5)
GCSScore                   42 /  7501	 32 /  725 (4.4)
AbdTenderDegree            12 /  5510	 30 / 1991 (1.5)
ThoracicTrauma              7 /  4746	  5 /  764 (0.7)
AbdomenPain                 5 /  4460	  2 /  286 (0.7)
DecrBreathSound             4 /  4438	  1 /   22 (4.5)
VomitWretch                 2 /  4094	  2 /  344 (0.6)
sensitivity 98.80
specificity 42.01

missed patients:


,Age,Sex,MOI
203,2,M,Fall down stairs
4349,2,M,Pedestrian/bicyclist struck by moving vehicle
